In [1]:
import numpy as np
import pandas as pd
from pgmpy.readwrite import XMLBIFReader
import random
import copy

# Settings
from discriminativeBN import DiscriminativeBN
from simplexMethod import NelderMeadOptimizer
from hookeJeeves import HookeJeevesOptimizer
from kfold_runner import k_fold_validation
from kfold_runner_lbfgs import k_fold_validation_gradient
from lbfgs import LBFGSOptimizerWrapper

/home/ernesto/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import time
import logging

# Set up basic logging to track failures
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# 1. SETUP: Define your Datasets, Models, and Target Variables here.
# IMPORTANT: Check "target_var" for diabetes, heart, etc. matches your CSV headers.
data_configs = [
    {    "name": "Australian",
        "csv": "datasets/AustralianDisc.csv",
        "xml": "models/australianNaive.xml",
        "target_var": "A15" 
    },
    {    "name": "Heart",
        "csv": "datasets/heartDisc.csv",
        "xml": "models/heartNaive.xml",
        "target_var": "class" 
    },
    {
        "name": "Diabetes",
        "csv": "datasets/diabetesDisc.csv",
        "xml": "models/diabetesNaive.xml",
        "target_var": "Class" 
    },
    {
        "name": "Pima",
        "csv": "datasets/pimaDisc.csv",
        "xml": "models/pimaNaive.xml",
        "target_var": "Class" 
    },
    {
        "name": "TicTacToe",
        "csv": "datasets/tic-tac-toe.csv",
        "xml": "models/ticTAcToeNaive.xml",
        "target_var": "Classs"
    }
]


# 2. SETUP: Define your Algorithms
optimizers = [
    {
        "name": "Nelder-Mead",
        "func": k_fold_validation,            # Standard function
        "Optimizer_fun": NelderMeadOptimizer,
        "max_iter": 6000,
        "file_suffix": "simplex"
    },
    {
        "name": "Hooke-Jeeves",
        "func": k_fold_validation,            # Standard function
        "Optimizer_fun": HookeJeevesOptimizer,
        "max_iter": 17,
        "file_suffix": "hooke"
    },
    {
        "name": "L-BFGS (Gradient)",
        "func": k_fold_validation_gradient,   # Gradient specific function
        "Optimizer_fun": LBFGSOptimizerWrapper,
        "max_iter": 100,
        "file_suffix": "LBFGS"
    }
]

# 3. BUILD: Generate the list of experiments dynamically
experiments = []

for data in data_configs:
    for opt in optimizers:
        
        # Auto-generate the results filename based on dataset and method
        # Example: results_AustralianTAN_simplex.txt
        result_name = f"results_{data['name']}NAIVE_{opt['file_suffix']}.txt"
        
        experiments.append({
            "name": f"{data['name']} - {opt['name']}",
            "func": opt['func'],
            "params": {
                "xml_file": data['xml'],
                "csv_file": data['csv'],
                "target_var": data['target_var'],
                "Optimizer_fun": opt['Optimizer_fun'],
                "max_iter": opt['max_iter'],
                "results_file": result_name
            }
        })

# 4. EXECUTE: Run the loop
print(f"Scheduled {len(experiments)} experiments.\n")

for exp in experiments:
    print(f"--- Starting Experiment: {exp['name']} ---")
    start_time = time.time()
    
    try:
        # Unpack the dictionary parameters into the function
        exp['func'](**exp['params'])
        
    except Exception as e:
        print(f"CRITICAL ERROR in {exp['name']}: {e}")
        logging.error(f"Failed {exp['name']}: {e}")
        # 'continue' jumps to the next iteration of the for-loop
        continue 
    
    else:
        # This only runs if the 'try' block succeeded
        end_time = time.time()
        elapsed_minutes = (end_time - start_time) / 60
        print(f"Successfully finished {exp['name']}.")
        print(f"Saved to: {exp['params']['results_file']}")
        print(f"Elapsed time: {elapsed_minutes:.2f} minutes\n")

print("\nAll scheduled experiments have been processed.")

Scheduled 15 experiments.

--- Starting Experiment: Australian - Nelder-Mead ---
Starting 10-Fold Validation on datasets/AustralianDisc.csv...
Optimizer: NelderMeadOptimizer | Max Iter: 6000
--- Starting Simplex Optimization (Dim: 60) ---
Initial Best Score: -426.4695 (CLL)
Initial Worst Score: -437.8572 (CLL)
------------------------------
CONVERGENCE ACHIEVED at Iter 1952.
Standard Deviation of Scores (0.099952) < Tolerance (0.1).
------------------------------
Optimization Finished. Best Score found: -186.4121
1 fold 85.51%
--- Starting Simplex Optimization (Dim: 60) ---
Initial Best Score: -457.4449 (CLL)
Initial Worst Score: -469.2180 (CLL)
------------------------------
CONVERGENCE ACHIEVED at Iter 1828.
Standard Deviation of Scores (0.099208) < Tolerance (0.1).
------------------------------
Optimization Finished. Best Score found: -193.4269
2 fold 86.96%
--- Starting Simplex Optimization (Dim: 60) ---
Initial Best Score: -398.9445 (CLL)
Initial Worst Score: -408.9479 (CLL)
----

In [3]:
data_configs = [
    {    "name": "Australian",
        "csv": "datasets/AustralianDisc.csv",
        "xml": "models/australianTAN.xml",
        "target_var": "A15" 
    },
    {    "name": "Heart",
        "csv": "datasets/heartDisc.csv",
        "xml": "models/heartTAN.xml",
        "target_var": "class" 
    },
    {
        "name": "Diabetes",
        "csv": "datasets/diabetesDisc.csv",
        "xml": "models/diabetesTAN.xml",
        "target_var": "Class" 
    },
    {
        "name": "Pima",
        "csv": "datasets/pimaDisc.csv",
        "xml": "models/pimaTAN.xml",
        "target_var": "Class" 
    },
    {
        "name": "TicTacToe",
        "csv": "datasets/tic-tac-toe.csv",
        "xml": "models/ticTAcToeTAN.xml",
        "target_var": "Classs"
    }
]

optimizers = [
    {
        "name": "Nelder-Mead",
        "func": k_fold_validation,            # Standard function
        "Optimizer_fun": NelderMeadOptimizer,
        "max_iter": 6000,
        "file_suffix": "simplex"
    },
    {
        "name": "Hooke-Jeeves",
        "func": k_fold_validation,            # Standard function
        "Optimizer_fun": HookeJeevesOptimizer,
        "max_iter": 17,
        "file_suffix": "hooke"
    },
    {
        "name": "L-BFGS (Gradient)",
        "func": k_fold_validation_gradient,   # Gradient specific function
        "Optimizer_fun": LBFGSOptimizerWrapper,
        "max_iter": 100,
        "file_suffix": "LBFGS"
    }
]